## Importing Packages and Modules

In [3]:
import transformers
import torch
import warnings
#Set to avoid warning messages.
transformers.logging.set_verbosity_error()
warnings.filterwarnings("ignore")

## Running the standard NER Pipeline

In [4]:
from transformers import pipeline

input_text="On 15th August 1947, India gained independence from British rule, \
with Jawaharlal Nehru becoming the first Prime Minister, and Delhi being declared the capital of the newly formed Republic of India"

basic_ner = pipeline("ner")

basic_ner(input_text)


[{'entity': 'I-LOC',
  'score': 0.99946374,
  'index': 6,
  'word': 'India',
  'start': 21,
  'end': 26},
 {'entity': 'I-MISC',
  'score': 0.9966717,
  'index': 10,
  'word': 'British',
  'start': 52,
  'end': 59},
 {'entity': 'I-PER',
  'score': 0.99846977,
  'index': 14,
  'word': 'J',
  'start': 71,
  'end': 72},
 {'entity': 'I-PER',
  'score': 0.9908835,
  'index': 15,
  'word': '##awa',
  'start': 72,
  'end': 75},
 {'entity': 'I-PER',
  'score': 0.9921595,
  'index': 16,
  'word': '##har',
  'start': 75,
  'end': 78},
 {'entity': 'I-PER',
  'score': 0.9961021,
  'index': 17,
  'word': '##lal',
  'start': 78,
  'end': 81},
 {'entity': 'I-PER',
  'score': 0.9992405,
  'index': 18,
  'word': 'Nehru',
  'start': 82,
  'end': 87},
 {'entity': 'I-LOC',
  'score': 0.9991875,
  'index': 26,
  'word': 'Delhi',
  'start': 127,
  'end': 132},
 {'entity': 'I-LOC',
  'score': 0.9986883,
  'index': 35,
  'word': 'Republic',
  'start': 180,
  'end': 188},
 {'entity': 'I-LOC',
  'score': 0.99468

## Understanding the model architecture

In [5]:
#Print model architecture
print(basic_ner.model)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

## Reviewing model configuration

In [6]:
print(basic_ner.model.config)


BertConfig {
  "_name_or_path": "dbmdz/bert-large-cased-finetuned-conll03-english",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_s

## Using a Custom Model and tokenizer

In [7]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER", 
                                          from_pt=True)

model = TFAutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER",
                                                          from_pt=True)

print(model.config.id2label)


{0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}


In [8]:
#Prediction
enhanced_ner = pipeline('ner', 
                        model=model, 
                        tokenizer=tokenizer, 
                        aggregation_strategy="simple")
enhanced_ner(input_text)

[{'entity_group': 'LOC',
  'score': 0.9997775,
  'word': 'India',
  'start': 21,
  'end': 26},
 {'entity_group': 'MISC',
  'score': 0.9996197,
  'word': 'British',
  'start': 52,
  'end': 59},
 {'entity_group': 'PER',
  'score': 0.999691,
  'word': 'J',
  'start': 71,
  'end': 72},
 {'entity_group': 'PER',
  'score': 0.9932279,
  'word': '##awa',
  'start': 72,
  'end': 75},
 {'entity_group': 'PER',
  'score': 0.91973585,
  'word': '##harlal Nehru',
  'start': 75,
  'end': 87},
 {'entity_group': 'LOC',
  'score': 0.9996086,
  'word': 'Delhi',
  'start': 127,
  'end': 132},
 {'entity_group': 'LOC',
  'score': 0.99853057,
  'word': 'Republic of India',
  'start': 180,
  'end': 197}]